<a href="https://colab.research.google.com/github/SaiSuvamPatnaik/Pytorch_Seq2Seq-Model/blob/main/Pytorch_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Note

In [137]:
embedding = nn.Embedding(4, 5)
input = torch.IntTensor([1,2,2,3])                # the values of your input all have to be less than or equal to num of embeddings
embedding(input)      

tensor([[-1.0394, -1.0253,  0.2044, -0.7960, -1.0917],
        [ 0.4117,  0.1532,  0.7218,  0.9184,  0.1444],
        [ 0.4117,  0.1532,  0.7218,  0.9184,  0.1444],
        [ 0.6666,  1.9112, -0.2977,  0.5994, -0.2920]],
       grad_fn=<EmbeddingBackward0>)

### Code

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

In [3]:
spacy_de = spacy.load('de_core_news_sm')          # Load the German tokeniser
spacy_en = spacy.load('en_core_web_sm')           # Load the English tokeniser

In [4]:
def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]         # Tokenizes German text from a string into a list of strings (tokens) and reverses it

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]               # Tokenizes English text from a string into a list of strings (tokens)

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  # exts specifies which languages to use as the source and target (source goes first) 
                                                    fields = (SRC, TRG))    # and fields specifies which field to use for the source and target.  

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


In [8]:
print(vars(train_data.examples[0]))   # Printing 1 Test example

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [9]:
SRC.build_vocab(train_data, max_size=10000, min_freq = 2)   # Using the min_freq argument, we only allow tokens that appear at least 2 times to appear in our vocabulary. 
TRG.build_vocab(train_data, max_size=10000, min_freq = 2)   # Tokens that appear only once are converted into an <unk> (unknown) token.

In [10]:
print(f"Unique tokens in source (de) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (de) vocabulary: 7855
Unique tokens in target (en) vocabulary: 5893


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [13]:
print(vars(train_iterator.data()[0]))

{'src': ['.', 'vordergrund', 'im', 'spielerin', 'einer', 'mit', 'roller-derby-spiel', 'einem', 'bei', 'actionfoto', 'ein'], 'trg': ['a', 'roller', 'derby', 'match', 'with', 'an', 'action', 'shot', 'of', 'a', 'derby', 'girl', 'in', 'the', 'foreground', '.']}


In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        embedded = self.dropout(self.embedding(src))                            # src = [src len, batch size]
        outputs, (hidden, cell) = self.rnn(embedded)                            # embedded = [src len, batch size, emb dim]
        return hidden, cell                                                     # outputs = [src len, batch size, hid dim * n directions]
                                                                                # hidden = [n layers * n directions, batch size, hid dim]
                                                                                # cell = [n layers * n directions, batch size, hid dim]
                                                                                # outputs are always from the top hidden layer        

In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)                                              # input = [batch size]
        embedded = self.dropout(self.embedding(input))                          # hidden = [n layers * n directions, batch size, hid dim]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))             # cell = [n layers * n directions, batch size, hid dim]
        prediction = self.fc_out(output.squeeze(0))                             # n directions in the decoder will both always be 1, therefore:
        return prediction, hidden, cell                                         # hidden = [n layers, batch size, hid dim]
                                                                                # context = [n layers, batch size, hid dim]
                                                                                # input = [1, batch size]
                                                                                # embedded = [1, batch size, emb dim]
                                                                                # output = [seq len, batch size, hid dim * n directions]
                                                                                # hidden = [n layers * n directions, batch size, hid dim]
                                                                                # cell = [n layers * n directions, batch size, hid dim]
                                                                                # seq len and n directions will always be 1 in the decoder, therefore:
                                                                                # output = [1, batch size, hid dim]
                                                                                # hidden = [n layers, batch size, hid dim]
                                                                                # cell = [n layers, batch size, hid dim]
                                                                                # prediction = [batch size, output dim]
        
        

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [17]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

In [18]:
enc, dec

(Encoder(
   (embedding): Embedding(7855, 256)
   (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
   (dropout): Dropout(p=0.5, inplace=False)
 ), Decoder(
   (embedding): Embedding(5893, 256)
   (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
   (fc_out): Linear(in_features=512, out_features=5893, bias=True)
   (dropout): Dropout(p=0.5, inplace=False)
 ))

In [19]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,899,013 trainable parameters


In [21]:
optimizer = optim.Adam(model.parameters())

In [22]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)


In [23]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()         #Setting the model in trainable form
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]

        output_dim = output.shape[-1]      # 5893

        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(epoch_loss)
    return epoch_loss / len(iterator)

In [24]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [25]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
vars(train_iterator.data()[0])   # Print item of train_iterator

{'src': ['.',
  'schellenkränze',
  'und',
  'rasseln',
  'spielen',
  'kirche',
  'einer',
  'keller',
  'im',
  'kinder',
  'gruppe',
  'eine'],
 'trg': ['a',
  'group',
  'of',
  'children',
  'in',
  'a',
  'church',
  'basement',
  'play',
  'maracas',
  'and',
  'tambourines',
  '.']}

In [27]:
for i,j in enumerate(train_iterator.data()):
  print(i,j.src,j.trg)
  break

0 ['.', 'ballbesitz', 'den', 'um', 'kämpfen', 'fußballspieler', 'drei'] ['three', 'soccer', 'players', 'are', 'battling', 'for', 'possession', 'of', 'the', 'ball', '.']


In [28]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(10):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')




    # print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    # print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    # print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

1151.957423210144
Epoch: 01 | Time: 1m 39s
	Train Loss: 5.075
	 Val. Loss: 5.256
1017.478081703186
Epoch: 02 | Time: 1m 39s
	Train Loss: 4.482
	 Val. Loss: 4.698
942.8974144458771
Epoch: 03 | Time: 1m 39s
	Train Loss: 4.154
	 Val. Loss: 4.610
890.2622718811035
Epoch: 04 | Time: 1m 39s
	Train Loss: 3.922
	 Val. Loss: 4.394
840.6969819068909
Epoch: 05 | Time: 1m 39s
	Train Loss: 3.704
	 Val. Loss: 4.199
802.0989599227905
Epoch: 06 | Time: 1m 39s
	Train Loss: 3.533
	 Val. Loss: 4.098
769.0467195510864
Epoch: 07 | Time: 1m 39s
	Train Loss: 3.388
	 Val. Loss: 3.964
735.8069462776184
Epoch: 08 | Time: 1m 40s
	Train Loss: 3.241
	 Val. Loss: 3.905
708.4693877696991
Epoch: 09 | Time: 1m 38s
	Train Loss: 3.121
	 Val. Loss: 3.824
679.3343758583069
Epoch: 10 | Time: 1m 39s
	Train Loss: 2.993
	 Val. Loss: 3.768


In [34]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.720 | Test PPL:  41.252 |


In [35]:
s1 = "tut mir leid"

In [36]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [64]:

def translate_sentence(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    spacy_ger = spacy.load("de")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # print(tokens)

    # sys.exit()
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]
    print(tokens)
    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    # Build encoder hidden, cell state
    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)

    outputs = [english.vocab.stoi["<sos>"]]
    
    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
        
        outputs.append(best_guess)

        # Model predicts it's the end of the sentence
        if output.argmax(1).item() == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]

    # remove start token
    return translated_sentence[1:]


In [65]:
translate_sentence(model,s1,SRC,TRG, device, max_length=50)

['<sos>', 'tut', 'mir', 'leid', '<eos>']


['lavender', 'striped', 'shirt', '.', '<eos>']

In [ ]:
!python -m spacy download de